In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Set inline
%matplotlib inline

In [2]:
def RCoeff(frequency, theta_i_degrees, cw, cp1, rho_ratio, rhow, alphap1):
    """
    Function to compute reflection coefficient.
    """
    # Convert attenuation to loss parameter
    deltap1 = alphap1 * cp1 * np.log(10) / (40 * np.pi * frequency)

    # Convert from degrees to radians
    theta_i = np.deg2rad(theta_i_degrees)

    # Complex sound speed
    cpp = cp1 / (1 + 1j * deltap1)

    # Ratio of sound speeds
    ap = cpp / cw

    # Impedance for water
    zw = rhow * cw / (np.sin(theta_i))

    # Impedance of sediment
    zp = rho_ratio * rhow * cpp / np.sqrt(1 - (ap * np.cos(theta_i))**2)

    # Impedance ratio
    zwp = zp / zw

    # Reflection coefficient
    R1 = (zwp - 1) / (zwp + 1)

    return R1

In [3]:
def perturb_roughness_scattering(frequency, theta_i_degrees, theta_s_degrees, phi_s, cw, cp1, rho_ratio, rhow, alphap1, w2, gamma2, L2):
    """
    Function to compute the roughness scattering cross section using perturbation theory"
    """
    # Convert attenuation to loss parameter
    deltap1 = alphap1 * cp1 * np.log(10) / (40 * np.pi * frequency)

    # Convert from degrees to radians
    theta_i = np.deg2rad(theta_i_degrees)
    theta_s = np.deg2rad(theta_s_degrees)

    # Ratio of sound speeds
    ap = cp1 / (cw * (1 + 1j * deltap1))

    # Wavenumber
    kw = 2 * np.pi * frequency / cw

    ## Reflection coefficients for theta_i and theta_s
    Vww_i = RCoeff(frequency, theta_i_degrees, cw, cp1, rho_ratio, rhow, alphap1)
    Vww_s = RCoeff(frequency, theta_s_degrees, cw, cp1, rho_ratio, rhow, alphap1)

    # sin(theta_{pi}) and sin(theta_{ps})
    stpi = np.sqrt(1 - (ap * np.cos(theta_i))**2)
    stps = np.sqrt(1 - (ap * np.cos(theta_s))**2)

    # Coefficients
    B = stpi * stps /(np.square(ap) * rho_ratio)

    G = (1 - 1 / rho_ratio) * (np.cos(theta_i) * np.cos(theta_s) * np.cos(phi_s) - B) - 1 + (1 / (np.square(ap) * rho_ratio))

    Aww = (1 + Vww_i) * (1 + Vww_s) * G / 2

    # incident wavenumber
    ki_x = kw * np.cos(theta_i)
    ki_y = 0;
    
    # scattered wavenumber
    ks_x = kw * np.cos(theta_s) * np.cos(phi_s)
    ks_y = kw * np.cos(theta_s) * np.sin(phi_s)

    # Bragg wavenumber
    dK = np.sqrt((ks_x - ki_x)**2 + (ks_y - ki_y)**2)

    # Spectral Level
    W_dK = w2 / (dK**2 + (2 * np.pi / L2)**2)**(gamma2 / 2)

    # Backscattering cross section
    sigma = (kw**4) * np.square(np.abs(Aww)) * W_dK

    return sigma

In [ ]:
def plot_scattering_strength(cp1, rhop1, alphap1, frequency):
    """
    # Function to plot magnitude and phase of the reflection coefficient and show critical / intromission angles.
    """

    # Water sound speed (m/s)
    cw = 1500

    # Water density (kg/m^3)
    rhow = 1000

    # Density ratio
    rho_ratio = rhop1 * 1000 / rhow

    # Grazing angles for reflection coefficient
    theta = np.arange(0.25, 60, 0.25)

    # Set phi_s = pi for backscatter
    phi_s = np.pi

    # Spectral Strength (m^(4-\gamma2))
    w2 = 0.0015

    # Spectral exponent
    gamma2 = 4.23

    # Cutoff Length (m)
    L2 = 0.2

    # 
    sigma = perturb_roughness_scattering(frequency * 1000, theta, theta, phi_s, cw, cp1, rho_ratio, rhow, alphap1, w2, gamma2, L2)

    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    plt.plot(theta, 10 * np.log10(sigma), linewidth=2)
    plt.xlabel('Grazing Angle (degrees)')
    plt.ylabel('Scattering Strength (dB)')
    plt.grid(True)
    
    plt.show



# Create interactive widgets
layout = widgets.Layout(width="500px", height="30px")
widgets.interact(
    plot_scattering_strength,
    cp1=widgets.FloatSlider(
        min=1300,
        max=1750,
        step=1,
        value=1720,
        description="Sediment Speed (m/s)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    rhop1=widgets.FloatSlider(
        min=1.2,
        max=2.5,
        step=0.1,
        value=2.1, 
        description="Sediment Density (g/cm^3)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    alphap1=widgets.FloatSlider(
        min=20,
        max=650,
        step=10,
        value=20, 
        description="Attenuation (dB/m)", 
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    frequency=widgets.FloatSlider(
        min=1,
        max=100,
        step=1,
        value=40, 
        description="Frequency (kHz)", 
        style={"description_width": "initial"}, 
        layout=layout,
    )
)

interactive(children=(FloatSlider(value=1720.0, description='Sediment Speed (m/s)', layout=Layout(height='30px…

<function __main__.plot_scattering_strength(cp1, rhop1, alphap1, frequency)>